# Classifying Logos in Images with Convolutionary Neural Networks

## Why should you read about machine learning and in particular this post?

The father of artificial intelligence (AI), John McCarthy, called AI "the science and engineering of making intelligent machines". Machine learning is the part of AI that is nowadays already reality. Here computers learn like humans from experience (= data) instead of being explicitly programmed.

As more and more data is available on which today's powerful computers can be trained, machine learning is now used everywhere: From customized video recommendations, image and speech recognition in medical research, spam and fraud detection, stock market analysis and video games to self-driving cars.

As a follow-up on [our review of the first term of Udacity's Machine Learning Engineer Nanodegree](https://medium.com/twodigits/https-medium-com-ajuschka-the-first-term-of-udacitys-machine-learning-engineer-nanodegree-592f14b586c), this blog post explains in detail how in our final deep learning project of the second term we implement Convolutional Neural Networks with Python's library Keras in order to classify logos in images. We are also interested in this topic as we later wish to analyze sentiments in twitter tweets with pictures where a given company's logo is detected.

## 1.  What image data do we use to train our Convolutional Neural Network?

Finding an interesting dataset is always the first step for any machine learning problem. Suited for our later purpose of classifying logos in arbitrary images from twitter tweets, we get access to the large [Logos in the Wild Dataset](https://www.iosb.fraunhofer.de/servlet/is/78045/) with 11,052 images containing different logo classes. Instead of downloading the images from the URLs provided with this dataset, we get 9,428 of them as a subset of the [QMUL-OpenLogo Dataset](https://www.iosb.fraunhofer.de/servlet/is/78045/). First, we load the images and label them with the target brand given by the corresponding folder name:

In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy 

#load files from LogosInTheWild directory with logo categories given by their subfolder name
data = load_files('LogosInTheWild-v2/data/voc_format')
logo_files = numpy.array(data['filenames'])
jpg_indices = [index for index, name in enumerate(logo_files) if 'jpg'==name.split('.')[-1]]
logo_targets = np_utils.to_categorical(numpy.array(data['target']), max(data['target']+1))
all_files, all_targets = logo_files[jpg_indices], logo_targets[jpg_indices]

After each training step, it makes sense to validate the resulting parameters and machine learning algorithm on a separate subset of the image files, the validation set. We define a checkpointer that saves the weight parameters with the best result on this validation set. In the end, after finishing the whole training process we want to test our machine learning algorithm on another separate subset, the test set. Thus we split our image dataset into training, validation and test set, and define the checkpointer  as follows:

In [ ]:
from sklearn.model_selection import train_test_split

#split into 80% training+validation files+labels, and 20 % test files+labels
train_and_val_files, test_files, train_and_val_targets, test_targets 
            = train_test_split(all_files, all_targets, test_size=0.2)
#further split into 80% training files+labels, and 20 % validation files+labels
train_files, val_files, train_targets, val_targets 
            = train_test_split(train_and_val_files, train_and_val_targets, test_size=0.2)
        
#create a checkpointer to save the CNN model with the best weight parameters
from keras.callbacks import ModelCheckpoint  
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               save_best_only=True)